## 0. Setup & Versions

In [ ]:


import os, sys, random, math, json, itertools, pathlib, time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

print('Python:', sys.version)
print('TensorFlow:', tf.__version__)
print('GPU Available:', tf.config.list_physical_devices('GPU'))


## 1. Config (Paths, Hyperparameters, Reproducibility)

In [ ]:
DATASET_ROOT = r"C:\Users\haris\Downloads\Multi_Crop_Dataset\dataset" # <-- Corrected Path
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_HEAD = 10
EPOCHS_FINETUNE = 10
FINE_TUNE_AT = 100
LEARNING_RATE = 1e-3
FINE_TUNE_LR = 1e-4
SEED = 42
SAVE_DIR = 'trained_models'

os.makedirs(SAVE_DIR, exist_ok=True)

import random
import numpy as np
import tensorflow as tf
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

## 2. Load Datasets (train/val/test)

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

full_train_ds, temp_val_ds = image_dataset_from_directory(
    DATASET_ROOT,
    validation_split=0.2,
    subset="both",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_ds_size = tf.data.experimental.cardinality(temp_val_ds)
val_ds = temp_val_ds.take(val_ds_size // 2)
test_ds = temp_val_ds.skip(val_ds_size // 2)

class_names = full_train_ds.class_names
num_classes = len(class_names)
print(f'Classes found: {class_names}')
print(f'Number of classes: {num_classes}')

AUTOTUNE = tf.data.AUTOTUNE
train_ds = full_train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)

print("\nSuccessfully created train, validation, and test datasets.")

## 3. Data Augmentation

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name='augmentation')


In [ ]:

def plot_history(history, title='Training History'):
    h = history.history
    plt.figure()
    plt.plot(h['accuracy'], label='train_acc')
    if 'val_accuracy' in h: plt.plot(h['val_accuracy'], label='val_acc')
    plt.title(title); plt.xlabel('epoch'); plt.ylabel('accuracy'); plt.legend(); plt.show()

    plt.figure()
    plt.plot(h['loss'], label='train_loss')
    if 'val_loss' in h: plt.plot(h['val_loss'], label='val_loss')
    plt.title(title); plt.xlabel('epoch'); plt.ylabel('loss'); plt.legend(); plt.show()


In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import seaborn as sns
import numpy as np

def evaluate_model(model, dataset, class_names, model_name='model', out_dir='trained_models'):
    y_true = []
    y_pred = []
    for images, labels in dataset:
        probs = model.predict(images, verbose=0)
        y_pred.extend(np.argmax(probs, axis=1).tolist())
        y_true.extend(labels.numpy().tolist())
    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True, zero_division=0)
    df_report = pd.DataFrame(report).transpose()
    acc = report['accuracy']

    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
    fig = plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=False, fmt='d', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix: {model_name}'); plt.ylabel('True'); plt.xlabel('Pred')
    fig_path = os.path.join(out_dir, f'{model_name}_confusion_matrix.png')
    fig.savefig(fig_path, bbox_inches='tight')
    plt.show()

    csv_path = os.path.join(out_dir, f'{model_name}_classification_report.csv')
    df_report.to_csv(csv_path, index=True)

    return acc, csv_path, fig_path


## 4. Model Builders

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers

def build_custom_cnn(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape + (3,))
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)
    # 6 conv + 5 pool
    for filters in [32, 64, 64, 128, 128, 256]:
        x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
        x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return keras.Model(inputs, outputs, name='CustomCNN')

def build_tl_model(base, input_shape, num_classes, name='TLModel'):
    base.trainable = False
    inputs = keras.Input(shape=input_shape + (3,))
    x = data_augmentation(inputs)
    x = layers.Rescaling(1./255)(x)
    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return keras.Model(inputs, outputs, name=name)

def compile_model(model, lr):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )


In [ ]:

from tensorflow.keras.applications import VGG16, MobileNetV2, DenseNet121

input_shape = (224, 224)
models = {}

models['CustomCNN'] = build_custom_cnn(input_shape, num_classes)

vgg = VGG16(weights='imagenet', include_top=False, input_shape=input_shape + (3,))
models['VGG16'] = build_tl_model(vgg, input_shape, num_classes, name='VGG16_TL')

mnet = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape + (3,))
models['MobileNetV2'] = build_tl_model(mnet, input_shape, num_classes, name='MobileNetV2_TL')

dnet = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape + (3,))
models['DenseNet121'] = build_tl_model(dnet, input_shape, num_classes, name='DenseNet121_TL')

for name, m in models.items():
    compile_model(m, 1e-3)
    print(f"\n{name} summary:")
    m.summary()


## 5. Train (Phase 1: Frozen base)

In [ ]:

callbacks = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]
histories = {}
val_accs_phase1 = {}

for name, model in models.items():
    print(f"\n===== Training {name} (frozen base) =====")
    h = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=callbacks, verbose=1)
    histories[name] = h
    plot_history(h, title=f'{name} (frozen)')
    val_accs_phase1[name] = max(h.history.get('val_accuracy', [0.0]))

val_accs_phase1


## 6. Fine-tune (Unfreeze top layers)

In [ ]:

def unfreeze_from(model, fine_tune_at=100):
    # Try to unfreeze top layers of the base submodel
    unfrozen = 0
    for layer in model.layers:
        if isinstance(layer, keras.Model):
            for i, bl in enumerate(layer.la yers):
                bl.trainable = (i >= fine_tune_at)
                if bl.trainable: unfrozen += 1
    print('Unfrozen layers:', unfrozen)

fine_tune_histories = {}
val_accs_phase2 = {}

for name, model in models.items():
    if name != 'CustomCNN':
        print(f"\n===== Fine-tuning {name} =====")
        unfreeze_from(model, fine_tune_at=100)
    else:
        print(f"\n===== Fine-tuning {name} (already trainable) =====")
    compile_model(model, 1e-4)
    h2 = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)], verbose=1)
    fine_tune_histories[name] = h2
    plot_history(h2, title=f'{name} (fine-tune)')
    val_accs_phase2[name] = max(h2.history.get('val_accuracy', [0.0]))

val_accs_phase2


## 7. Evaluate on Test & Save

In [ ]:

import json, os

results = []
best_name = None
best_val = -1.0

for name, model in models.items():
    print(f"\n===== Test Evaluation: {name} =====")
    test_acc, report_csv, cm_png = evaluate_model(model, test_ds, class_names, model_name=name, out_dir='trained_models')
    best_val_acc = max(val_accs_phase2.get(name, 0.0), val_accs_phase1.get(name, 0.0))
    if best_val_acc > best_val:
        best_val = best_val_acc
        best_name = name
    model_path = os.path.join('trained_models', f'{name}.h5')
    model.save(model_path)
    results.append({'name': name, 'val_best': float(best_val_acc), 'test_acc': float(test_acc),
                    'report_csv': report_csv, 'confusion_matrix_png': cm_png, 'model_path': model_path})

with open(os.path.join('trained_models', 'results.json'), 'w') as f:
    json.dump({'results': results, 'best_model': best_name}, f, indent=2)

print('\nSummary:')
for r in sorted(results, key=lambda x: x['val_best'], reverse=True):
    print(f"{r['name']}: val_best={r['val_best']:.4f}, test_acc={r['test_acc']:.4f}")
print('Best model (by val):', best_name)


## 8. Quick Inference (single image)

In [ ]:

import json, os
from tensorflow import keras
import numpy as np

with open(os.path.join('trained_models', 'results.json'), 'r') as f:
    info = json.load(f)
best_name = info['best_model']
best_model_path = [r['model_path'] for r in info['results'] if r['name'] == best_name][0]
best_model = keras.models.load_model(best_model_path)
print('Loaded best model:', best_name)

def predict_image(img_path, img_size=(224,224)):
    img = keras.utils.load_img(img_path, target_size=img_size)
    x = keras.utils.img_to_array(img)[None, ...] / 255.0
    probs = best_model.predict(x, verbose=0)[0]
    idx = int(np.argmax(probs))
    return class_names[idx], float(probs[idx])

# Example:
label, conf = predict_image(r'Downloads/Multi_Crop_Dataset/dataset/tomato/Tomato_Early_blight/0abc57ec-7f3b-482a-8579-21f3b2fb780b___RS_Erly.B 7609.JPG')
print(label, conf)
